In [1]:
import numpy as np
import pandas as pd
import recsys as rs
import json
import notipy
import random
import TopSimilarRecommender as TSR

In [2]:
train = pd.read_csv('Data/train_final.csv','\t')
tr_info = pd.read_csv('Data/tracks_final.csv','\t')
tgt_pl = pd.read_csv('Data/target_playlists.csv','\t')
tgt_tr = pd.read_csv('Data/target_tracks.csv','\t')

In [3]:
big_target_playlist = [i for i in np.unique(train['playlist_id'].values) if train[train['playlist_id'] == i].shape[0] >= 10]

In [4]:
test_pl = pd.DataFrame({"playlist_id": list(map(lambda x: random.choice(big_target_playlist), range(100)))})

In [5]:
tracks_in_test_pl = pd.merge(train, test_pl, how='inner', on='playlist_id')
tracks_removed= pd.DataFrame(columns=['playlist_id', 'track_id'], dtype='int32')
indexes_to_remove= np.array([], dtype='int32')
for i in np.unique(tracks_in_test_pl['playlist_id'].values):
    #randomly selects 5 elements from playlist i and save indexes
    tmp = (tracks_in_test_pl.where(tracks_in_test_pl['playlist_id']==i).dropna().sample(5)).index
    #insert the items selected in tracks_removed dataFrame
    tracks_removed= tracks_removed.append(tracks_in_test_pl.take(tmp)).astype('int64')
    #remove tracks from the original dataframe
    indexes_to_remove=np.append(indexes_to_remove, values=tmp)

tracks_in_train= pd.DataFrame(tracks_in_test_pl)
tracks_in_train.drop(indexes_to_remove, inplace=True)
tracks_removed

,playlist_id,track_id
3009,156430,1818779
3023,156430,341446
3019,156430,1945113
2995,156430,573087
3013,156430,2351541
1964,326993,1147854
1972,326993,4526860
1969,326993,1989002
1966,326993,1739287
1968,326993,2954157


In [6]:
tgt_test_tracks= pd.DataFrame(tracks_in_test_pl['track_id'])
print(tgt_test_tracks.shape)
tgt_test_tracks = pd.DataFrame.drop_duplicates(tgt_test_tracks)
print(tgt_test_tracks.shape)

(4500, 1)
(4208, 1)


In [8]:
tracks_in_test_pl

,playlist_id,track_id
0,10083808,1989513
1,10083808,1255654
2,10083808,336690
3,10083808,2386151
4,10083808,2910266
5,10083808,1052122
6,10083808,1670042
7,10083808,2524197
8,10083808,368688
9,10083808,905773


In [9]:
fit_dict = {'tracks_info' : tr_info, 
            'attributes' : ['artist_id', 'album', 'tags'],
            'tgt_tracks' : tgt_test_tracks,
            'n_min_attr' : 2,
            'measure' : 'dot',
            'shrinkage' : 0,
            'n_el_sim' : 20}

recommend_dict = {'tgt_playlists' : test_pl,
                  'train_playlists_tracks_pairs' : tracks_in_test_pl, 
                  'normalize' : False}

In [10]:
rec = TSR.TopSimilarRecommender()
rec.fit(**fit_dict)
notipy.notify('Model fitted!')
print('Model fitted!')

Fixed dataset
Calculated Indices
Pruning attributes.
Number of attributes before pruning is: 120295
Number of attributes after pruning is: 35250
ICM built
Computed 0 similarities over 100000 with dot measure and 0 shrinkage.
Computed 1000 similarities over 100000 with dot measure and 0 shrinkage.
Computed 2000 similarities over 100000 with dot measure and 0 shrinkage.
Computed 3000 similarities over 100000 with dot measure and 0 shrinkage.
Computed 4000 similarities over 100000 with dot measure and 0 shrinkage.
Computed 5000 similarities over 100000 with dot measure and 0 shrinkage.
Computed 6000 similarities over 100000 with dot measure and 0 shrinkage.
Computed 7000 similarities over 100000 with dot measure and 0 shrinkage.
Computed 8000 similarities over 100000 with dot measure and 0 shrinkage.
Computed 9000 similarities over 100000 with dot measure and 0 shrinkage.
Computed 10000 similarities over 100000 with dot measure and 0 shrinkage.
Computed 11000 similarities over 100000 with

In [11]:
recommendetions = rec.recommend(**recommend_dict)
notipy.notify('Recommending completed!')
print('Recommending completed!')

Calculated 0 users ratings over 100
URM built
Recommended 0 users over 100


SystemExit: Not enough similarity

/Users/LucaButera/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [23]:
map_eval = rs.evaluate(R, tracks_removed, 'MAP')
notipy.notify('Evaluation completed!')
print('Evaluation completed!')

ZeroDivisionError: ('float division by zero', 'occurred at index 0')

In [ ]:
run_data = {'recommender_type' : rec.__class__.__name__,
            'fit_parameters' : {'attributes' : fit_dict['attributes'],
                                'n_min_attr' : fit_dict['n_min_attr'],
                                'measure' : fit_dict['measure'],
                                'shrinkage' : fit_dict['shrinkage'],
                                'n_el_sim' : fit_dict['n_el_sim']},
            'recommend_parameters' : {'normalize' : recommend_dict['normalize']},
            'evaluation_result' : map_eval}

In [ ]:
with open('runs_data.json', 'w') as fp:
    json.dump(run_data, fp)
notipy.notify('Run data saved!')
print('Run data saved!')